In [6]:
import cairosvg
import numpy as np
import cv2
from collections import Counter

In [8]:
# Function to print colored box in console
def color_box(rgb):
    r, g, b = rgb
    return f"\033[48;2;{r};{g};{b}m  \033[0m"

In [19]:
# Path to any SVG
svg_path = "../4x3_flags/ad.svg"

# Convert SVG to PNG (in memory)
png_bytes = cairosvg.svg2png(url=svg_path)

# Decode PNG bytes into an OpenCV image
img_array = np.frombuffer(png_bytes, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)

print(f"Image shape: {img.shape}")

# Convert from BGR to RGB for more intuitive colors
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Reshape into list of pixels
pixels = img_rgb.reshape(-1, 3)

# Convert to tuples so they can be counted
pixels_tuples = [tuple(pixel) for pixel in pixels]

# Count color occurrences
color_counts = Counter(pixels_tuples)
total_pixels = len(pixels_tuples)

# Threshold: colors that occupy >=5% of the image
threshold = 0.05 * total_pixels
dominant_colors = {
    color: count for color, count in color_counts.items() if count >= threshold
}

# Display results
print("\nDominant colors (>=5% of image):")
for color, count in dominant_colors.items():
    pct = 100 * count / total_pixels
    swatch = color_box(color)
    print(f"{swatch} RGB{color} — {pct:.2f}%")

Image shape: (480, 640, 3)

Dominant colors (>=5% of image):
   RGB(np.uint8(0), np.uint8(24), np.uint8(168)) — 31.88%
   RGB(np.uint8(254), np.uint8(223), np.uint8(0)) — 27.67%
   RGB(np.uint8(208), np.uint8(16), np.uint8(58)) — 31.88%


In [22]:
import os
import json
import pandas as pd
from tabulate import tabulate

svg_dir = "../4x3_flags"
json_path = "../flag_list_json/sovereign_plus_few_flags.json"

with open(json_path, "r") as f:
    country_map = json.load(f)

rows = []
table_rows = []

# Loop through all SVGs
for filename in os.listdir(svg_dir):
    if not filename.endswith(".svg"):
        continue

    country_code = os.path.splitext(filename)[0]
    if country_code not in country_map:
        continue  # skip other codes

    country_name = country_map[country_code]
    svg_path = os.path.join(svg_dir, filename)

    # Convert SVG to PNG (in memory)
    png_bytes = cairosvg.svg2png(url=svg_path)
    img_array = np.frombuffer(png_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)

    # Convert BGR to RGB
    if img.shape[2] == 4:
        img = img[:, :, :3]  # drop alpha if present
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten pixels and count colors
    pixels = img_rgb.reshape(-1, 3)
    pixels_tuples = [tuple(pixel) for pixel in pixels]
    color_counts = Counter(pixels_tuples)
    total_pixels = len(pixels_tuples)

    # Keep colors >=5% of image
    threshold = 0.05 * total_pixels
    dominant_colors = [color for color, count in color_counts.items() if count >= threshold]
    dominant_colors = dominant_colors[:6]  # limit to 6

    # Prepare row for DataFrame
    row = [country_name] + dominant_colors
    row += [None] * (6 - len(dominant_colors))  # pad to 6 colors
    rows.append(row)

    # Prepare row for table (convert colors to swatches)
    table_row = [country_name] + [color_box(c) for c in dominant_colors]
    table_row += [""] * (6 - len(dominant_colors))
    table_rows.append(table_row)

# Create DataFrame
columns = ["Country"] + [f"Color{i+1}" for i in range(6)]
df = pd.DataFrame(rows, columns=columns)

# Print pretty table
print("\nPretty table with color boxes:")
print(tabulate(table_rows, headers=columns, tablefmt="grid"))

# Optional: preview DataFrame
print("\nDataFrame preview:")
print(df.head())


Pretty table with color boxes:
+----------------------------------+----------+----------+----------+----------+----------+----------+
| Country                          | Color1   | Color2   | Color3   | Color4   | Color5   | Color6   |
+==================================+==========+==========+==========+==========+==========+==========+
| Andorra                          |          |          |          |          |          |          |
+----------------------------------+----------+----------+----------+----------+----------+----------+
| United Arab Emirates             |          |          |          |          |          |          |
+----------------------------------+----------+----------+----------+----------+----------+----------+
| Afghanistan                      |          |          |          |          |          |          |
+----------------------------------+----------+----------+----------+----------+----------+----------+
| Antigua and Barbuda              |     

In [23]:
df.to_csv('flag_dominant_colours.csv', index=False)